In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
from scipy.linalg import pascal
from scipy.stats import norm
import warnings
import scipy
warnings.filterwarnings("ignore")

In [3]:
S_0, K, B = 100, 80, 160
r, sigma = 0.2, 0.3
T, periods = 1, 50
t_grid = np.linspace(0, T, periods)

In [4]:
def BSM(s, r, sigma, t1, t2, strike):
    dt = t2 - t1
    return r * s * dt + sigma * s * np.random.normal(0, np.sqrt(dt))

In [5]:
T, N = 1, 7 * 10 ** 2
S_0, B, K, M = 100, 300, 200, 10 ** 2
r, sigma= 0.1, 0.05
T_wave = (sigma ** 2) * T / 2
D = (2 * r) / (sigma ** 2)
X_1 = np.log(S_0 / K)
X_2 = np.log(B / K)
X_arr = np.linspace(X_1, X_2, M)
tau = T_wave / N
h = abs(X_1 - X_2) / M
print(tau / ((h ** 2) / 2))
U_shape = (N, M)
print('U_shape:', N * M)
U = np.zeros(U_shape)

0.029590551774650833
U_shape: 70000


In [6]:
D = (2 * r) / (sigma ** 2)

for m in range(len(X_arr)):
    if 1 - np.exp(-X_arr[m]) > 0:
        U[0][m] = 1 - np.exp(-X_arr[m])
    else:
        U[0][m] = 0

for n in range(N):
    U[n, -1] = 1 - np.exp(-D * tau * n - X_arr[-1])

In [7]:
def find_u(u_left, u_mid, u_right, tau, h, D):
    # нахождение значения в узле
    ans = u_mid + (tau / (h ** 2)) * (u_left - 2 * u_mid + u_right) + ((tau / h) * (1 + D)) * (u_right - u_mid)
    if ans > 0:
        return ans
    return 0

In [8]:
# поиск значений на сетке
for n in tqdm(range(N - 1)):
    for m in range(1, M - 1):
        U[n + 1, m] = find_u(U[n, m - 1], U[n, m], U[n, m + 1], tau, h, D)
V = U * np.linspace(S_0, B, M)
V_numerical = pd.DataFrame(V)
ans_numerical = V_numerical.sum(axis=0) / V_numerical.shape[0]


100%|██████████| 699/699 [00:00<00:00, 4053.16it/s]


In [9]:
def delta_plus(s, r, sigma, T, t):
    '''
    for barrier call options
    '''
    tau = T - t 
    return (1 / (sigma * np.sqrt(tau))) * (np.log(s) + (r + pow(sigma, 2) / 2) * tau)

In [10]:
def delta_minus(s, r, sigma, T, t):
    '''
    for barrier call options
    '''
    tau = T - t 
    return (1 / (sigma * np.sqrt(tau))) * (np.log(s) + (r - pow(sigma, 2) / 2) * tau)

In [11]:
def barrier_options_exact(s, r, sigma, T, t, K, B):
    '''
    0 <= t <= T
    s = s(t)
    0 < s <= B / s
    '''
    tau = T - t
    first_part = s * (norm.cdf(delta_plus(s / K, r, sigma, T, t)) - norm.cdf(delta_plus(s / B, r, sigma, T, t)))
    second_part = np.exp(-tau * r) * K * (norm.cdf(delta_minus(s / K, r, sigma, T, t)) - norm.cdf(delta_minus(s / B, r, sigma, T, t)))
    third_part = -B * ((s / B) ** (-2 * r / pow(sigma, 2))) * (norm.cdf(delta_plus(pow(B, 2) / (K * s), r, sigma, T, t)) - norm.cdf(delta_plus(B / s, r, sigma, T, t)))
    fourth_part = np.exp(-r * tau) * K * ((s / B) ** (-2 * r / pow(sigma, 2) + 1)) * (norm.cdf(delta_minus(pow(B, 2) / (K * s), r, sigma, T, t)) - norm.cdf(delta_minus(pow(B, 2) / s, r, sigma, T, t)))
    price = first_part - second_part - third_part + fourth_part
    if price <= 0:
        return 0
    return price

In [12]:
B_arr = K * np.exp(X_arr)
t_arr = T - (2 / (sigma ** 2)) * np.linspace(0, T_wave, N)

In [13]:
exact = np.zeros((U_shape))
for i in tqdm(range(len(t_arr))):
    for j in range(len(B_arr)):
        exact[i][j] = barrier_options_exact(B_arr[j], sigma, T, t_arr[i], r, K, B)

100%|██████████| 700/700 [01:02<00:00, 11.23it/s]
